## Introduction
This model will manage the database code for course descriptor.


In [4]:
# Install MySQL connector
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 32.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


In [5]:
import numpy as np
import mysql
import mysql.connector as msql
from mysql.connector import Error
from datetime import datetime

In [6]:
def get_db_connection(userName, Password):
  # establish database connection
  try:
    conn = msql.connect(host='db4free.net',database='education_nz' ,user=userName,
                          password=Password)
  except Error as e:
      print("Error while connecting to MySQL now ", e)

  return conn

In [7]:
def set_connection_close(conn):
  # Close the cursor and connection
  if conn.is_connected():
    conn.close()

In [8]:
def log_error(conn, errorTitle, error_message):

  # Get the current date and time
  now = datetime.now()

  # Format the current date and time as a string in MySQL datetime format
  dt_string = now.strftime('%Y-%m-%d %H:%M:%S')

  try :
    if conn.is_connected():
      cursor = conn.cursor()

      # SQL query to insert data into the 'error' table
      insert_query = 'INSERT INTO education_nz.error (title, description, creation_datetime) VALUES (%s, %s, %s)'
      values =(errorTitle, error_message, dt_string )

      cursor.execute(insert_query, values)
      conn.commit()

  except Error as e:
      print("Error while inserting to error table", e)
  finally:
    # Close the cursor and connection
    if conn.is_connected():
      cursor.close()


In [9]:
def insert_version(conn, name, des):

  # Get the current date and time
  now = datetime.now()

  # Format the current date and time as a string in MySQL datetime format
  dt_string = now.strftime('%Y-%m-%d %H:%M:%S')

  try:
    if conn.is_connected():
      cursor = conn.cursor()

      insert_query = 'INSERT INTO education_nz.version (name, description, created_datetime) VALUES (%s, %s, %s)'
      values =(name, des, dt_string )

      cursor.execute(insert_query, values)
      conn.commit()

      # Retrieve the last inserted ID
      version_id = cursor.lastrowid
  except Error as e:
      title = "Error while inserting to version table"
      log_error(conn, title, str(e))

  finally:
    # Close the cursor and connection
    if conn.is_connected():
      cursor.close()


  return version_id

In [10]:
def insert_version(conn, name, des):

  version_id = None

  # Get the current date and time
  now = datetime.now()

  # Format the current date and time as a string in MySQL datetime format
  dt_string = now.strftime('%Y-%m-%d %H:%M:%S')

  try:
    if conn.is_connected():
      cursor = conn.cursor()

      insert_query = 'INSERT INTO education_nz.version (name, description, created_datetime) VALUES (%s, %s, %s)'
      values =(name, des, dt_string )

      cursor.execute(insert_query, values)
      conn.commit()

      # Retrieve the last inserted ID
      version_id = cursor.lastrowid
  except Error as e:
      title = "Error while inserting to version table"
      log_error(conn, title, str(e))

  finally:
    # Close the cursor and connection
    if conn.is_connected():
      cursor.close()


  return version_id

In [11]:
def insert_course(conn, code, title, credit, level, self_directed_learning_hrs,
                  tutor_directed_learning_hrs, aim, versionID, pageNo):

  # Placeholder for the returned courseID
  course_id = None

  try:

      if conn.is_connected():
          cursor = conn.cursor()

          # SQL query to insert data into the 'course' table
          insert_query = """
              INSERT INTO course (
                  code, title, credit, level, self_directed_learning_hrs,
                  tutor_directed_learning_hrs, aim, versionID, pageNo
              ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
          """

          # Execute the query with the provided data
          cursor.execute(insert_query, (
              code, title, credit, level, self_directed_learning_hrs,
              tutor_directed_learning_hrs, aim, versionID, pageNo
          ))

          # Commit the changes
          conn.commit()

          # Retrieve the last inserted ID
          course_id = cursor.lastrowid

  except Error as e:
      title = "Error while inserting to course table"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()

  return course_id



In [12]:
def get_courseID_by_criteria(conn, code, title, versionID):

  course_id = None

  try:

      if conn.is_connected():
          cursor = conn.cursor(buffered=True)

          # SQL query to retrieve courseID based on matching criteria
          select_query = """
              SELECT courseID FROM course
              WHERE (code = %s OR title = %s) AND versionID = %s
          """

          # Execute the query with the provided data
          cursor.execute(select_query, (code, title, versionID))

          # Fetch the result
          result = cursor.fetchone()

          if result:
              course_id = result[0]
          else:
              print("No matching record found.")

  except Error as e:
      title = "Error while retrieving courseID"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()

  return course_id




def get_versionID_by_courseID(conn, courseID):

  version_id = None

  try:

      if conn.is_connected():
          cursor = conn.cursor(buffered=True)

          # SQL query to retrieve versionID based on courseID
          select_query = """
              SELECT versionID FROM course WHERE courseID = %s
          """

          # Execute the query with the provided data
          cursor.execute(select_query, ([courseID]))

          # Fetch the result
          result = cursor.fetchone()

          if result:
              version_id = result[0]
          else:
              print("No matching record found for courseID.")

  except Error as e:
      title = "Error while retrieving versionID"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()

  return version_id



In [13]:
def insert_prerequisite(conn, code, title, courseID):

  # Placeholder for the returned prerequisiteID
  prerequisite_id = None

  try:

      if conn.is_connected():
          cursor = conn.cursor()

          # SQL query to insert data into the 'prerequisite' table
          insert_query = """
              INSERT INTO prerequisite (code, title, courseID) VALUES (%s, %s, %s)
          """

          # Execute the query with the provided data
          cursor.execute(insert_query, (code, title, courseID))

          # Commit the changes
          conn.commit()

          # Retrieve the last inserted ID
          prerequisite_id = cursor.lastrowid

  except Error as e:
      title = "Error while inserting to prerequisite table"
      log_error(conn, title, str(e))

  finally:
        # Close the cursor and connection
      if conn.is_connected():
        cursor.close()


  return prerequisite_id

def update_prerequisite_courseID(conn, prerequisiteID, prerequisite_courseID):

    try:
        if conn.is_connected():
            cursor = conn.cursor()

            # SQL query to update prerequisite_courseID based on prerequisiteID
            update_query = """
                UPDATE prerequisite SET prerequisite_courseID = %s WHERE prerequisiteID = %s
            """

            # Execute the update query with the provided data
            cursor.execute(update_query, (prerequisite_courseID, prerequisiteID))

            # Commit the changes
            conn.commit()

    except Error as e:
            title = "Error while updating prerequisite_courseID"
            log_error(conn, title, str(e))

    finally:
        # Close the cursor and connection
        if conn.is_connected():
          cursor.close()



def get_null_prerequisites(conn):

  null_prerequisites = []

  try:

      if conn.is_connected():
          cursor = conn.cursor(dictionary=True, buffered=True)

          # SQL query to retrieve records where prerequisite_courseID is NULL
          select_query = """
              SELECT prerequisiteID, code, title, courseID
              FROM prerequisite
              WHERE prerequisite_courseID IS NULL
          """

          # Execute the query
          cursor.execute(select_query)

          # Fetch all the results
          null_prerequisites = cursor.fetchall()


  except Error as e:
      title = "Error while retrieving null prerequisites"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()

  return null_prerequisites


In [14]:
def insert_completion(conn, criteria, courseID):

  try:
      if conn.is_connected():
          cursor = conn.cursor()

          # SQL query to insert data into the completion table
          insert_query = """
              INSERT INTO completion (criteria, courseID)
              VALUES (%s, %s)
          """

          # Execute the query with the provided data
          cursor.execute(insert_query, (criteria, courseID))

          # Commit the changes
          conn.commit()

  except Error as e:
      title = "Error while inserting to completion table"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()



In [15]:
def insert_outcome(conn, outcome_Index, outcome, courseID):

  try:
      if conn.is_connected():
          cursor = conn.cursor()

          # SQL query to insert data into the outcome table
          insert_query = """
              INSERT INTO outcome (outcome_Index, outcome, courseID)
              VALUES (%s, %s, %s)
          """

          # Execute the query with the provided data
          cursor.execute(insert_query, (outcome_Index, outcome, courseID))

          # Commit the changes
          conn.commit()

  except Error as e:
      title = "Error while inserting to outcome table"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()



In [16]:
def insert_assessment(conn, method, weight_percentage, courseID):

  assessment_id = None

  try:
      if conn.is_connected():
          cursor = conn.cursor()

          # SQL query to insert data into the assessment table
          insert_query = """
              INSERT INTO assessment (method, weight_precentage, courseID)
              VALUES (%s, %s, %s)
          """

          # Execute the query with the provided data
          cursor.execute(insert_query, (method, weight_percentage, courseID))

          # Commit the changes
          conn.commit()

          # Retrieve the last inserted ID (assessmentID)
          assessment_id = cursor.lastrowid

  except Error as e:
      title = "Error while inserting to assessment table"
      log_error(conn, title, str(e))
  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()

  return assessment_id



In [17]:
def insert_assessment_outcome(conn, assessmentID, outcomeID_value):

  try:
      if conn.is_connected():
          cursor = conn.cursor()

          # SQL query to insert data into the assessment_outcome table
          insert_query = """
              INSERT INTO assessment_outcome (assessmentID, outcomeID_value)
              VALUES (%s, %s)
          """

          # Execute the query with the provided data
          cursor.execute(insert_query, (assessmentID, outcomeID_value))

          # Commit the changes
          conn.commit()

  except Error as e:
      title = "Error while inserting to assessment_outcome table"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()



In [18]:
def find_assessments_with_null_outcome(conn):
  try:

      if conn.is_connected():
          cursor = conn.cursor(dictionary=True, buffered=True)

          # SQL query to find assessments with null outcomeID
          select_query = """
              SELECT a.assessmentID, ao.outcomeID_value, a.courseID
              FROM assessment a
              LEFT JOIN assessment_outcome ao ON a.assessmentID = ao.assessmentID
              WHERE ao.outcomeID IS NULL
          """

          # Execute the query
          cursor.execute(select_query)

          # Fetch all records
          result = cursor.fetchall()

          return result

  except Error as e:
      title = "Error while finding assessments with null outcome"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()


In [19]:
def get_outcomeID_by_index_and_course(conn, outcome_index, courseID):

  outcome_id = None

  try:

      if conn.is_connected():
          cursor = conn.cursor(dictionary=True, buffered=True)

          # SQL query to get outcomeID by outcome_index and courseID
          select_query = """
              SELECT outcomeID
              FROM outcome
              WHERE outcome_Index = %s AND courseID = %s
          """

          # Execute the query with the provided data
          cursor.execute(select_query, (outcome_index, courseID))

          # Fetch the result
          result = cursor.fetchone()

          if result:
              outcome_id = result['outcomeID']

  except Error as e:
      title = "Error while getting outcomeID by outcome_index and courseID"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()

  return outcome_id



In [20]:
def update_outcomeID(conn, assessmentID, outcomeID_value, new_outcomeID):

  try:

      if conn.is_connected():
          cursor = conn.cursor()

          # SQL query to update outcomeID in assessment_outcome
          update_query = """
              UPDATE assessment_outcome
              SET outcomeID = %s
              WHERE assessmentID = %s AND outcomeID_value = %s
          """

          # Execute the update query with the provided data
          cursor.execute(update_query, (new_outcomeID, assessmentID, outcomeID_value))

          # Commit the changes
          conn.commit()

  except Error as e:
      title = "Error while updating outcomeID"
      log_error(conn, title, str(e))

  finally:
      # Close the cursor and connection
      if conn.is_connected():
        cursor.close()

